# [실습 프로젝트] Naive RAG 구현 

- RAG 프롬프트 구성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

# Test

## 1. Setting

1\) 환경 변수 로드

In [ ]:
from dotenv import load_dotenv
load_dotenv()

2\) 문서 로드

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('./data/bart.pdf')
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

3\) 데이터 확인

In [ ]:
# 문단이 분리된 경우에도 문장 순서를 올바르게 읽는지 확인
pdf_docs[0].page_content[2200:2300]

In [ ]:
# 도표로 시작하는 페이지 데이터 확인
pdf_docs[1].page_content[:100]

## 2. Embedding

1\) Text Split

- **Semantic Chunking** 방식으로 텍스트를 분할함<br>
    → 임베딩 벡터 간의 **기울기(gradient)** 변화를 기준으로 의미 단위(semantic unit)를 구분함<br>
    → 청크 길이에 일관성이 없으며, 문맥에 따라 길이가 유동적으로 결정됨

- 길이가 100자 미만인 청크는 이미지 기반 텍스트(OCR 등)로 간주하여 제거함<br>
    → 주요 텍스트가 아닌 부가 정보일 가능성이 높기 때문임

- 1차 분할된 청크는 길이 편차가 크므로, 문자열 길이 기준으로 재귀적으로 분할하여 최종적으로는 일관된 길이의 청크를 구성함

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker 
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 임계값 타입 설정 (gradient, percentile, standard_deviation, interquartile)
)
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

In [ ]:
selected_chunks = []
for idx, chunk in enumerate(chunks):
    content = chunk.page_content
    if len(chunk.page_content) < 100:
        print(f'{idx}: {content}')
    else:
        selected_chunks.append(chunk)

print(f"생성된 청크 수: {len(selected_chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in selected_chunks)}")

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,                      
    chunk_overlap=100,
    separators=[" \n", ".\n", ". "],
)
final_chunks = text_splitter.split_documents(selected_chunks)
print(f"생성된 텍스트 청크 수: {len(final_chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in final_chunks)}")

2\) Embedding
- 문서 임베딩 도구로 `OpenAIEbeddings` 선택함

In [ ]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    dimensions=1024
)
documents = [chunk.page_content for chunk in final_chunks]
document_embeddings_openai = embeddings_model.embed_documents(documents)
print(f"임베딩 벡터의 개수: {len(document_embeddings_openai)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_openai[0])}")
print(document_embeddings_openai[0])

In [ ]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(
        query: str, 
        documents: list,
        doc_embeddings: np.ndarray,
        embeddings_model
    ) -> tuple[str, float]:
    
    # 쿼리 임베딩: OpenAI 임베딩 사용 
    query_embedding = embeddings_model.embed_query(query)

    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # 가장 유사한 문서 인덱스 찾기
    most_similar_idx = np.argmax(similarities)

    # 가장 유사한 문서와 유사도 반환: 문서, 유사도
    return documents[most_similar_idx], similarities[most_similar_idx]


query = "What is BART architecture?"
most_similar_doc, similarity = find_most_similar(
    query, 
    documents,
    document_embeddings_openai, 
    embeddings_model=embeddings_model
    )
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {most_similar_doc}")
print(f"유사도: {similarity:.4f}")

## 3. Save Vector

- Vectorstore로 `ChromaDB` 사용함

In [ ]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="my_collection2",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

In [ ]:
chroma_db.get()

In [ ]:
# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(len(final_chunks))]

# 문서를 벡터 저장소에 저장
added_doc_ids = chroma_db.add_documents(documents=final_chunks, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

## 4. Retriever

- **MMR** 검색으로 상위 3개 문서 검색하는 Retriever 사용함
- **Cosine similarity** 를 사용하여 임베딩 품질을 확인함

In [ ]:
chroma_mmr = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.3,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)

In [ ]:
# 검색 테스트 
query = "What is BART architecture?"
retrieved_docs = chroma_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_model.embed_query(query)], 
        [embeddings_model.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]} \n[유사도: {score}]")
    print("-" * 100)

## 5. Prompt

In [ ]:
# Prompt 템플릿 (예시)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

translate_prompt = ChatPromptTemplate.from_template(
    "Translate the following into English: {query}"
)
work_prompt = ChatPromptTemplate.from_template("""
Please answer following these rules:
1. Answer the questions based only on [Context].
2. If there is no [Context], answer that you don't know.
3. Do not use external knowledge.
4. If there is no clear basis in [Context], answer that you don't know.
5. You can refer to the previous conversation.

[Context]
{context}

[Question] 
{question}

[Answer]
""")
output_prompt = ChatPromptTemplate.from_template(
    "Translate the following into Korean: {output}"
)

## 6. Chain

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.8,
    top_p=0.7
)
output_parser = StrOutputParser()

# 문서 포맷팅
def format_docs(docs):
    """ 참고 문서 연결 """
    return "\n\n".join([f"{i}: \n{doc.page_content}" for i, doc in enumerate(docs)])

def format_result(answer):
    """ 최종 응답 처리 """
    output = answer['output']
    context = answer['context']
    return f"{output}\n\n[Context]\n{context}"

# 체인 생성
translate_chain = translate_prompt | llm | output_parser
rag_chain = chroma_mmr | RunnableLambda(format_docs)
output_chain = work_prompt | llm | output_parser | output_prompt | llm | output_parser

main_chain = (
    translate_chain |
    RunnableParallel(
        question=RunnablePassthrough(),
        context=lambda x: rag_chain.invoke(x),
    ) | 
    RunnableParallel(
        context=lambda x: x['context'],
        output=output_chain
    ) | RunnableLambda(format_result)
)

In [ ]:
query = "BART의 강점이 모야?"
answer = main_chain.invoke({"query": query})
print(f"쿼리: {query}")
print("답변:")
print(answer)

# Chat Interface

- `Gradio`를 활용하여 Chat Interface를 구현함
- 위에서 테스트한 내용을 기능별로 정리하여 구현함

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnableMap, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

`(1) 벡터 저장소 설정`

- **text-embedding-3-small** 임베딩 모델을 활용하여 **Chroma** 벡터 저장소를 사용함 

In [3]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    dimensions=1024
)

In [4]:
from langchain_chroma import Chroma

chroma_db = Chroma(
    collection_name="my_collection2",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

`(2) 검색기 정의`

- MMR 검색으로 상위 3개 문서를 검색하는 Retriever 사용함

In [5]:
def format_docs(docs):
    """ 참고 문서 연결 """
    return "\n\n".join([f"{i}: \n{doc.page_content}" for i, doc in enumerate(docs)])

In [6]:
def get_context_chain(question):
    """ Retriever """
    chroma_mmr = chroma_db.as_retriever(
        search_type='mmr',
        search_kwargs={
            'k': 3,
            'fetch_k': 8,
            'lambda_mult': 0.3,
        },
    )
    chain = chroma_mmr | RunnableLambda(format_docs)
    return chain.invoke(question)

`(3) RAG 프롬프트 구성`

In [7]:
def to_english_chain(model):
    prompt = ChatPromptTemplate.from_template(
        "Translate the following into English: {query}"
    )
    return prompt | model | StrOutputParser()

def to_korean_chain(model):
    prompt = ChatPromptTemplate.from_template(
        "Translate the following into Korean: {query}"
    )
    return prompt | model | StrOutputParser()

def get_anwser_chain(model):
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder("chat_history"),
        ("human", """
Please answer following these rules:
1. Answer the questions based only on [Context].
2. If there is no [Context], answer that you don't know.
3. Do not use external knowledge.
4. If there is no clear basis in [Context], answer that you don't know.
5. You can refer to the previous conversation.

[Context]
{context}

[Question] 
{question}

[Answer]
"""
    )])
    return prompt | model | StrOutputParser()

`(4) RAG 체인 구성`

- 대화 히스토리는 영문으로 작성된 내용만 저장 및 활용함

In [8]:
from typing import Iterator
from operator import itemgetter

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.8,
    top_p=0.7
)
memory_store = []

In [10]:
def update_memory(x):
    memory_store.append(HumanMessage(content=x["question"]))
    memory_store.append(AIMessage(content=x["answer"]))
    return f"{x['korean_answer']}\n\n[Context]\n{x['context']}"

In [11]:
def get_streaming_response(message: str, history) -> Iterator[str]:
    translate_chain = to_english_chain(model)
    korean_chain = to_korean_chain(model)
    answer_chain = get_anwser_chain(model)

    full_chain = (
        translate_chain |
        RunnableMap({
            "question": RunnablePassthrough(),  # English question
            "context": lambda q: get_context_chain(q),  # get_context는 이미 함수로 있음
            "chat_history": RunnableLambda(lambda _: memory_store)
        }) |
        RunnableMap({
            "question": itemgetter("question"),
            "context": itemgetter("context"),
            "query": answer_chain
        }) |
        RunnableMap({
            "question": itemgetter("question"),
            "context": itemgetter("context"),
            "answer": itemgetter("query"),
            "korean_answer": korean_chain
        }) |
        RunnableLambda(update_memory)
    )
    return full_chain.invoke(message)

In [12]:
import gradio as gr

# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=get_streaming_response,         # 메시지 처리 함수
    title="BART에 대해", # 채팅 인터페이스의 제목
    type="messages"
)

# 실행
demo.launch()

/Users/sunhwaryu/Documents/llm-study/prj01/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [13]:
# demo 실행 종료
demo.close()

Closing server running on port: 7860


In [14]:
memory_store

[HumanMessage(content='What is it that makes BART better than BERT?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Based on the [Context], BART generalizes BERT by using a bidirectional encoder like BERT and a left-to-right decoder like GPT, combining these features in a Transformer-based sequence-to-sequence architecture. Additionally, BART's decoder layers perform cross-attention over the encoder's final hidden layer, which BERT does not have. BART also contains roughly 10% more parameters than an equivalently sized BERT model. These architectural differences, along with its training method of corrupting and reconstructing text, contribute to BART achieving better performance on various tasks compared to BERT.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Explain the strengths of BART.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Based on the [Context], the strengths of BART include:\n\n- It significantly outperforms p